<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Catboost</a></span></li><li><span><a href="#SDG" data-toc-modified-id="SDG-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>SDG</a></span></li></ul></li><li><span><a href="#Тестирование-лучшей-модели" data-toc-modified-id="Тестирование-лучшей-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование лучшей модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем все необходимые библиотеки, откроем наш датасет, изучим его.

In [1]:
#pip install pymystem3

In [2]:
#%pip install -U pip setuptools wheel
#%pip install -U spacy

In [3]:
#!python -m spacy download en_core_web_sm

In [4]:
#!pip install imblearn

In [5]:
import numpy as np
import re
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from pymystem3 import Mystem
import spacy
from imblearn.under_sampling import RandomUnderSampler
from sklearn.utils import shuffle
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm
from sklearn.pipeline import Pipeline

In [6]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [7]:
data.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [8]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [10]:
data.duplicated().sum()

0

Удалим столбец *Unnamed*, который не несет никакой смысловой нагрузки.

In [11]:
data = data.drop('Unnamed: 0', axis=1)

In [12]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим баланс классов.

In [13]:
data.toxic.value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [14]:
data.toxic.value_counts()/len(data)*100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

Мы обнаружили дисбаланс классов: негативных комментариев почти в 9 раз меньше. В дальнейшем придется сбалансировать выборку.

Лемматизируем наш текст с помощью *spacy*, напишем функцию лемматизации, затем применим ее к тексту. На выходе получаем практически готовые признаки.

In [15]:
nlp = spacy.load('en_core_web_sm', disable = ['ner', 'parser'])

In [16]:
def lemmatize(text):
    nlp_text = nlp(text)
    lemm_text = " ".join([i.lemma_ for i in nlp_text])
    final_text = re.sub(r'[^a-zA-Z ]', ' ', lemm_text)
    return " ".join(final_text.split())

In [17]:
tqdm.pandas()

data['lemm_text'] = data['text'].progress_apply(lemmatize)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [18]:
data.head(20)

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D aww he match this background colour I be see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...
5,"""\n\nCongratulations from me as well, use the ...",0,congratulation from I as well use the tool wel...
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,COCKSUCKER before you pis around on MY work
7,Your vandalism to the Matt Shirvington article...,0,your vandalism to the Matt Shirvington article...
8,Sorry if the word 'nonsense' was offensive to ...,0,sorry if the word nonsense be offensive to you...
9,alignment on this subject and which are contra...,0,alignment on this subject and which be contrar...


Разделим исходные данные на обучающую и тестовую выборки в соотношении 90:10. После чего, выделим признаки и целевой признак для каждой выборки.

In [19]:
target = data['toxic']
features = data['lemm_text']

data_train, data_test = train_test_split(data, test_size=0.10, random_state=12345)

features_train = data_train['lemm_text']
target_train = data_train['toxic']
features_test = data_test['lemm_text']
target_test = data_test['toxic']

In [20]:
print('Размер тренировочной выборки:', data_train.shape[0])
print('Размер тестовой выборки:', data_test.shape[0])

Размер тренировочной выборки: 143362
Размер тестовой выборки: 15930


Создадим матрицу со значениями TF-IDF с помощью TfidfVectorizer(). Перед этим создадим счётчик, указав в нём стоп-слова.

In [23]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
features_train_tfidf = count_tf_idf.fit_transform(features_train)
features_test_tfidf = count_tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gleb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
print(features_train_tfidf.shape)
print(features_train_tfidf.shape)

(143362, 147784)
(143362, 147784)


Наши данные полностью готовы к обучению моделей.

## Обучение

Для подбора гиперпараметров для наших моделей будем использовать Pipeline. Начнем с решающего дерева.

### Дерево решений

In [25]:
parameters_dt = {'model_dt__max_depth': range(1, 10, 2),
                 'model_dt__min_samples_leaf': range(1, 3),
                 'model_dt__min_samples_split': range(2,5,2)}

In [26]:
pipe_dt = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('model_dt', DecisionTreeClassifier(random_state=12345))])

In [27]:
%%time

model_dt = GridSearchCV(pipe_dt, param_grid=parameters_dt, n_jobs=-1, scoring='f1')
model_dt.fit(features_train, target_train)
print(model_dt.best_params_)
print(model_dt.best_score_)

{'model_dt__max_depth': 9, 'model_dt__min_samples_leaf': 1, 'model_dt__min_samples_split': 4}
0.5926758450653015
Wall time: 3min 15s


### Случайный лес

In [28]:
parameters_rf = {'model_rf__n_estimators': range (10, 31, 10),
                 'model_rf__max_depth': range (1, 8, 2),
                 'model_rf__min_samples_leaf': range (1,4),
                 'model_rf__min_samples_split': range (2,7,2) }

In [29]:
pipe_rf = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('model_rf', RandomForestClassifier(random_state=12345))])

In [30]:
%%time

model_rf = GridSearchCV(pipe_rf, param_grid=parameters_rf, n_jobs=-1, scoring='f1')
model_rf.fit(features_train, target_train)
print(model_rf.best_params_)
print(model_rf.best_score_)

{'model_rf__max_depth': 7, 'model_rf__min_samples_leaf': 2, 'model_rf__min_samples_split': 6, 'model_rf__n_estimators': 10}
0.0002743954805450263
Wall time: 13min 21s


### Логистическая регрессия

In [31]:
parameters_lr = {'model_lr__penalty': [None, 'l1', 'l2', 'elasticnet'],
                 'model_lr__C': [0.05, 0.1, 0.5, 1, 5, 10]}

In [32]:
pipe_lr = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('model_lr', LogisticRegression(random_state=12345))])

In [33]:
%%time

model_lr = GridSearchCV(pipe_lr, param_grid=parameters_lr, n_jobs=-1, scoring='f1')
model_lr.fit(features_train, target_train)
print(model_lr.best_params_)
print(model_lr.best_score_)

C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
90 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, sel

{'model_lr__C': 10, 'model_lr__penalty': 'l2'}
0.776659007114958
Wall time: 2min 52s


C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Catboost

In [34]:
parameters_cb = {'model_cb__depth'         : [3,5,7],
                 'model_cb__learning_rate' : [0.01,0.02,0.03],
                 'model_cb__iterations'    : [10,20]}

In [35]:
pipe_cb = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('model_cb', CatBoostClassifier(verbose=False))])

In [36]:
%%time

model_cb = GridSearchCV(pipe_cb, param_grid=parameters_cb, n_jobs=-1, scoring='f1', verbose=False)
model_cb.fit(features_train, target_train)
print(model_cb.best_params_)
print(model_cb.best_score_)

C:\Users\Gleb\anaconda3\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
19 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
19 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\pipe

{'model_cb__depth': 5, 'model_cb__iterations': 20, 'model_cb__learning_rate': 0.03}
0.4622265296561688
Wall time: 18min 49s


### SDG

In [37]:
model_sdg = SGDClassifier(random_state=12345)

In [38]:
parameters_sdg = {'model_sdg__loss':['hinge', 'log', 'modified_huber'],
                  'model_sdg__learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'],
                  'model_sdg__eta0':[0.01, 0.05, 0.1, 0.2, 0.3, 0.5]}

In [39]:
pipe_sdg = Pipeline([('vectorizer', TfidfVectorizer(stop_words=stopwords)),
                    ('model_sdg', SGDClassifier(random_state=12345))])

In [40]:
%%time

model_sdg = GridSearchCV(pipe_sdg, param_grid=parameters_sdg, n_jobs=-1, scoring='f1')
model_sdg.fit(features_train, target_train)
print(model_sdg.best_params_)
print(model_sdg.best_score_)

{'model_sdg__eta0': 0.2, 'model_sdg__learning_rate': 'constant', 'model_sdg__loss': 'modified_huber'}
0.743788309096648
Wall time: 9min 27s


Лучшей моделью оказалась логистическая регрессия, при параметрах: *'C': 10, 'penalty': 'l2'*;  **f1=0.7766**. Проверим нашу модель на тестовой выборке.

## Тестирование лучшей модели

In [42]:
%%time

model = LogisticRegression(random_state=12345, C = 10, penalty = 'l2')
model.fit(features_train_tfidf, target_train)
pred = model.predict(features_test_tfidf)
f1 = f1_score(target_test, pred)
f1

Wall time: 5.13 s


C:\Users\Gleb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7911479944674966

## Выводы

В ходе текущей работы мы:

- открыли файл с данными, изучили его, выполнили подготовку данных;
- выявили дисбаланс классов целевого признака;
- лемматизировали наш текст с помощью spacy;
- перевели наш текст в матрицу с помощью TfidfVectorizer();
- обучили ряд моделей, используя GridSearchCV и Pipeline.
- выявили наилучшую модель, подобрав наилучшие гиперпараметры, ей оказалась логистическая регрессия при параметрах C = 10, penalty = 'l2';
- протестировали нашу модель на тестовой выборке, рассчитали метрику f1 и получили удовлетворительный результат **f1=0.79**.